In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import model_from_json

# Charger l'architecture
with open("modele/modele.json", "r") as f:
    modele_json = f.read()
modele = model_from_json(modele_json)

# Charger les poids
modele.load_weights("modele/modele.weights.h5")
print("Modèle chargé.")


In [ ]:
LONGUEUR_IMAGE = 28
LARGEUR_IMAGE = 28

observations_test = pd.read_csv('datas/fashion-mnist_test.csv')
X_test = np.array(observations_test.iloc[:, 1:])
y_test = np.array(observations_test.iloc[:, 0])

def prep(x):
    x = x.reshape(x.shape[0], LARGEUR_IMAGE, LONGUEUR_IMAGE, 1).astype('float32')
    x /= 255.0
    return x

X_test_img = prep(X_test)

classes = ["Un T-shirt/haut","Un pantalon","Un pull","Une robe","Un manteau",
           "Des sandales","Une chemise","Des baskets","Un sac","Des bottes de cheville"]


In [ ]:
idx = [0, 1, 2]   # 3 exemples
probas = modele.predict(X_test_img[idx])
preds = np.argmax(probas, axis=1)

for i, p in zip(idx, preds):
    print(f"Image {i} -> prédiction: {classes[p]} (label réel: {y_test[i]})")

In [ ]:
from PIL import Image, ImageFilter

def load_and_preprocess_image(image_path, target_size=28, content_size=20):
    image = Image.open(image_path).convert('L')
    # redimensionne sans déformer
    w, h = image.size
    if w > h:
        new_w, new_h = content_size, max(1, round(content_size * h / w))
    else:
        new_h, new_w = content_size, max(1, round(content_size * w / h))
    resized = image.resize((new_w, new_h), Image.LANCZOS).filter(ImageFilter.SHARPEN)
    # centre sur un canevas blanc 28x28
    canvas = Image.new('L', (target_size, target_size), 255)
    x_off = (target_size - new_w)//2
    y_off = (target_size - new_h)//2
    canvas.paste(resized, (x_off, y_off))
    # inverse fond blanc -> noir et normalise
    arr = (255 - np.array(canvas).astype('float32'))/255.0
    return arr.reshape(1, target_size, target_size, 1)

img = load_and_preprocess_image("images/mon_image.png")
proba = modele.predict(img)[0]
pred = np.argmax(proba)
print(f"Selon moi l'image est : {classes[pred]}")
